<a href="https://colab.research.google.com/github/SergioPedroso/Modelo_de_classificacao_dex_fantastic-sniffle/blob/main/Day_7_Modelo_de_Recomendacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Importando bibliotecas

In [23]:
import pandas as pd
import numpy as np
import seaborn as sns
pd.options.display.float_format = "{:,.2f}".format


### Importando arquivos

In [28]:
df_order_items = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Slot #6/Day 7/DNC_order_items_dataset.csv",index_col=0)
df_order_reviews = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Slot #6/Day 7/DNC_order_reviews_dataset.csv",index_col=0)
df_orders = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Slot #6/Day 7/DNC_orders_dataset.csv",index_col=0)
df_products = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Slot #6/Day 7/DNC_products_dataset.csv",index_col=0)

In [29]:
df_order_items.head()

,order_id,order_item_id,product_id,price
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,58.90
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,239.90
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,199.00
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,12.99
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,199.90


In [30]:
df_order_reviews.head()

,review_id,order_id,review_score
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5


In [31]:
df_orders.head()

,order_id,customer_id,order_status
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered


In [32]:
df_products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.00,287.00,225.00,16.00,10.00,14.00
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.00,276.00,"1,000.00",30.00,18.00,20.00
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.00,250.00,154.00,18.00,9.00,15.00
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.00,261.00,371.00,26.00,4.00,26.00
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.00,402.00,625.00,20.00,17.00,13.00


### EDA

In [33]:
print("df_order_items: ",df_order_items.shape)
print("df_order_reviews: ",df_order_reviews.shape)
print("df_orders: ",df_orders.shape)
print("df_products: ",df_products.shape)

df_order_items:  (112650, 4)
df_order_reviews:  (99224, 3)
df_orders:  (99441, 3)
df_products:  (32951, 8)


In [54]:
df_order_items.describe(include='number',percentiles=[.10,.25,.50,.75,.90,.99])

,order_item_id,price
count,"112,650.00","112,650.00"
mean,1.20,120.65
std,0.71,183.63
min,1.00,0.85
10%,1.00,23.80
25%,1.00,39.90
50%,1.00,74.99
75%,1.00,134.90
90%,2.00,229.80
99%,4.00,890.00


In [34]:
df_products.describe(include='all',percentiles=[.10,.25,.50,.75,.90,.99])

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm
count,32951,32341,"32,341.00","32,341.00","32,949.00","32,949.00","32,949.00","32,949.00"
unique,32951,73,nan,nan,nan,nan,nan,nan
top,5dfc5bdca7ee27155face9d4e89201ae,cama_mesa_banho,nan,nan,nan,nan,nan,nan
freq,1,3029,nan,nan,nan,nan,nan,nan
mean,NaN,NaN,48.48,771.50,"2,276.47",30.82,16.94,23.20
std,NaN,NaN,10.25,635.12,"4,282.04",16.91,13.64,12.08
min,NaN,NaN,5.00,4.00,0.00,7.00,2.00,6.00
10%,NaN,NaN,33.00,207.00,175.00,16.00,4.00,11.00
25%,NaN,NaN,42.00,339.00,300.00,18.00,8.00,15.00
50%,NaN,NaN,51.00,595.00,700.00,25.00,13.00,20.00


In [46]:
print(df_order_items.isnull().sum(),'\n')
print(df_order_reviews.isnull().sum(),'\n')
print(df_orders.isnull().sum(),'\n')
print(df_products.isnull().sum(),'\n')

order_id         0
order_item_id    0
product_id       0
price            0
dtype: int64 

review_id       0
order_id        0
review_score    0
dtype: int64 

order_id        0
customer_id     0
order_status    0
dtype: int64 

product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64 



### Data Cleaning

In [50]:
df_products2 = df_products.dropna(axis=0, how='any')
print(df_products2.isnull().sum(),'\n')

product_id                    0
product_category_name         0
product_name_lenght           0
product_description_lenght    0
product_weight_g              0
product_length_cm             0
product_height_cm             0
product_width_cm              0
dtype: int64 



In [62]:
df_order_items_aux = df_order_items.groupby('order_id').agg({
                        'order_item_id':"sum",
                        'product_id':'sum',
                        'price':'mean',
}) 
df_order_items_aux.columns = ['_'.join(col).strip() if col[1] != "" else col[0] for col in df_order_items_aux.columns.values]

In [52]:
df_order_aux = df_order_items.merge(df_order_reviews,how='left',on=['order_id'])


,order_id,order_item_id,product_id,price,review_id,review_score
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,58.90,97ca439bc427b48bc1cd7177abe71365,5.00
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,239.90,7b07bacd811c4117b742569b04ce3580,4.00
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,199.00,0c5b33dea94867d1ac402749e5438e8b,5.00
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,12.99,f4028d019cb58564807486a6aaf33817,4.00
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,199.90,940144190dcba6351888cafa43f3a3a5,5.00
...,...,...,...,...,...,...
113309,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,299.99,9185f849f32d82e216a4e025e0c50f5c,5.00
113310,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,350.00,be803f6a93d64719fd685c1cc610918a,5.00
113311,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,99.90,dbdd81cd59a1a9f94a10a990b4d48dce,5.00
113312,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,55.99,fba117c9ac40d41ca7be54741f471303,5.00


In [63]:
df_order_aux.nunique()

order_id         98666
order_item_id       21
product_id       32951
price             5968
review_id        97709
review_score         5
dtype: int64